In [2]:
import pandas as pd 
import numpy as np
import re
import os
import argparse
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import SimpleRNN , Dropout ,LSTM,Conv1D, GlobalMaxPooling1D, MaxPooling1D , SpatialDropout1D,Activation,Dense, Dropout, Activation, Flatten, Input, concatenate , GRU
from keras.layers.embeddings import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('arabic')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
dataset1 = pd.read_csv('../input/arab-data/dataarabTOx.csv')
dataset6 = pd.read_csv('../input/arab-data/Book1.csv')
new = dataset6.dropna()
# print(new.iloc[10])
# print(new.iloc[9])
dataset7 = pd.read_csv('../input/arab-data/Book2.csv')
dataset = pd.read_csv('../input/data-set/toxic arabic tweets classification - Copy.tsv' , delimiter = '\t' , quoting = 3)
dataset2 = pd.read_csv('../input/arabic-data/OSACT2020-sharedTask-dev.tsv' , delimiter = '\t' , quoting = 3)
dataset3 = pd.read_csv('../input/arabic-data/OSACT2020-sharedTask-train.tsv' , delimiter = '\t' , quoting = 3)
dataset4 =dataset2.iloc[:,0:2]
dataset5 =dataset3.iloc[:,0:2]
df_concat = pd.concat([dataset5,dataset,dataset4,new,dataset7,dataset1])
for i in range(0,len(df_concat)):
    if df_concat['Class'].iloc[i] == 'normal' or df_concat['Class'].iloc[i] == 'NOT_OFF' or df_concat['Class'].iloc[i] == 'Non-Offensive' or df_concat['Class'].iloc[i] == 0:
        df_concat['Class'].iloc[i] = 'N'
    elif df_concat['Class'].iloc[i] == 'abusive' or df_concat['Class'].iloc[i] == 'hate' or df_concat['Class'].iloc[i] == 'OFF' or df_concat['Class'].iloc[i] == 'Offensive' or df_concat['Class'].iloc[i] == -1 or df_concat['Class'].iloc[i] == -2:
        df_concat['Class'].iloc[i] = 'P'
print(df_concat.shape)

(32377, 2)


In [5]:
X = df_concat.iloc[: , 0].values
y = df_concat.iloc[: , -1].values
from keras.callbacks import ModelCheckpoint ,ReduceLROnPlateau ,EarlyStopping

early = EarlyStopping(monitor='val_loss', mode='min', patience=4) 
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
callbacks_list = [early, learning_rate_reduction]

In [6]:
labelencoder_y=LabelEncoder()
y = labelencoder_y.fit_transform(y)
for i in range(0,len(y)):
    if y[i]==0 or y[i]==1:
        continue
    else:
        y[i]=0
print(max(y))

1


In [7]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42)

In [8]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u2066"
        u"\u2069"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', str(data))

def to_lowercase(text):
    return text.lower()

def remove_eng(text):
    return re.sub(r'[A-Za-z]','',str(text))
    
import string
def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):

    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_emojis(text)
    text = remove_eng(text)
    text = remove_punctuation(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
X_train = normalize_corpus(X_train)
X_test = normalize_corpus(X_test)

In [10]:
tok = Tokenizer()
tok.fit_on_texts(X_train + X_test)
vocab_size = len(tok.word_index) + 1
encoded_docs_train = tok.texts_to_sequences(X_train)
encoded_docs_test = tok.texts_to_sequences(X_test)
#encoded_docs_check = tok.texts_to_sequences(nor_check)

max_length = 300
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
#padded_docs_check = pad_sequences(encoded_docs_check, maxlen=max_length, padding='post')
print(padded_docs_train)
print(np.shape(padded_docs_train))


[[  110  2515  9214 ...     0     0     0]
 [18845     0     0 ...     0     0     0]
 [    6  5006     0 ...     0     0     0]
 ...
 [28788 69726  1598 ...     0     0     0]
 [  151  1273   445 ...     0     0     0]
 [    3  3397    33 ...     0     0     0]]
(25901, 300)


In [11]:
inputs1 = Input(shape=(max_length,))
embedding1 = Embedding(vocab_size, 50, input_length=max_length)(inputs1)
conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling1D(pool_size=2)(drop1)
flat1 = Flatten()(pool1)
# channel 2
inputs2 = Input(shape=(max_length,))
embedding2 = Embedding(vocab_size, 50, input_length=max_length)(inputs2)
conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
drop2 = Dropout(0.5)(conv2)
pool2 = MaxPooling1D(pool_size=2)(drop2)
flat2 = Flatten()(pool2)
# channel 3
inputs3 = Input(shape=(max_length,))
embedding3 = Embedding(vocab_size, 50, input_length=max_length)(inputs3)
conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
drop3 = Dropout(0.5)(conv3)
pool3 = MaxPooling1D(pool_size=2)(drop3)
flat3 = Flatten()(pool3)
# merge
merged = concatenate([flat1, flat2, flat3])
# interpretation
dense1 = Dense(10, activation='relu')(merged)
outputs = Dense(1, activation='sigmoid')(dense1)
model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
# compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# summarize
model.summary()

2022-05-28 14:24:57.072372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 300, 50)      3991950     input_1[0][0]                    
______________________________________________________________________________________________

2022-05-28 14:24:57.231233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 14:24:57.232064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 14:24:57.233239: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-28 14:24:57.233566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [12]:
history = model.fit([padded_docs_train,padded_docs_train,padded_docs_train], y_train , validation_split = 0.2 , epochs=10 , batch_size = 32, callbacks=callbacks_list)

2022-05-28 14:25:05.909846: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-05-28 14:25:07.728335: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


648/648 [==============================] - 15s 10ms/step - loss: 0.5174 - accuracy: 0.7452 - val_loss: 0.3966 - val_accuracy: 0.8332
Epoch 2/10
648/648 [==============================] - 7s 10ms/step - loss: 0.1682 - accuracy: 0.9354 - val_loss: 0.3882 - val_accuracy: 0.8253
Epoch 3/10
648/648 [==============================] - 7s 10ms/step - loss: 0.0379 - accuracy: 0.9886 - val_loss: 0.4394 - val_accuracy: 0.8313
Epoch 4/10
648/648 [==============================] - 7s 10ms/step - loss: 0.0145 - accuracy: 0.9957 - val_loss: 0.5167 - val_accuracy: 0.8195

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 5/10
648/648 [==============================] - 7s 10ms/step - loss: 0.0081 - accuracy: 0.9979 - val_loss: 0.5497 - val_accuracy: 0.8340
Epoch 6/10
648/648 [==============================] - 6s 10ms/step - loss: 0.0067 - accuracy: 0.9979 - val_loss: 0.5752 - val_accuracy: 0.8342

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.00000042747

In [13]:
loss, accuracy = model.evaluate([padded_docs_test,padded_docs_test,padded_docs_test], y_test)
print('Accuracy: %f' % (accuracy*100),'  ',loss)

203/203 [==============================] - 1s 3ms/step - loss: 0.5768 - accuracy: 0.8355
Accuracy: 83.554661    0.5767640471458435


In [16]:
import keras
while True:
        
        inp = input("User: ")
        
        if inp.lower() == "quit":
            break
        else:
            z = keras.preprocessing.sequence.pad_sequences(tok.texts_to_sequences(normalize_corpus([inp])),maxlen=300, padding='post')
            #result1 = model1.predict([z,z,z])
            result = model.predict([z,z,z])
            #avg = (result1+result2)/2
            print(result)
            dis = np.argmax(result)
            if result <= 0.5:
                print("normal")
            else:
                print("abusive")

User:  


[[0.32507703]]
normal


User:  kneklvn


[[0.32507703]]
normal


User:  الحمد لله


...

User:  انا تعبت منك


[[0.11133708]]
normal


User:  اخرس شويه


[[0.25727665]]
normal


User:  quit
